In [149]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Embedding, Flatten, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

import pandas as pd

In [150]:
df = pd.read_csv('spam_or_not_spam.csv')


In [151]:
df.head()
#label 0 = ham, 1 = spam

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [152]:
df = df.dropna() #delete colomn who have null values. note: wasting so many time because of this
df.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [153]:
label = LabelEncoder().fit_transform(df['label'])
kalimat = df['email']
label

array([0, 0, 0, ..., 1, 1, 1])

In [154]:
x_train, x_test, y_train, y_test = train_test_split(kalimat, label, test_size=0.2, random_state =1)

In [155]:

#kodekan kata-kata ke angka
tokenizer = Tokenizer(num_words=2000, oov_token='ukv')
tokenizer.fit_on_texts(x_train) 

#ubah kata ke kode angka
sekuens_latih = tokenizer.texts_to_sequences(x_train) 
sekuens_test = tokenizer.texts_to_sequences(x_test)
 
padded_latih = pad_sequences(sekuens_latih, maxlen =100) 
padded_test = pad_sequences(sekuens_test, maxlen = 100)

padded_latih.shape[0]

2399

In [162]:
x_train = padded_latih
x_test = padded_test

In [157]:
#x_train = padded_latih.reshape(padded_latih.shape[0], 1, padded_latih.shape[1]) #input LSTM BUTUH 3 Dimensi
#x_test = padded_test.reshape(padded_test.shape[0], 1, padded_test.shape[1]) #(jumlah batch, time step, panjang sequence)

In [163]:
model = Sequential([
    Embedding(input_dim =2000, output_dim=100),
    LSTM(200, input_shape = (1,100), return_sequences = True), #return_sequences = mengembalikan input terakhir kedalam sequence tidak
    LSTM(100, activation='sigmoid', return_sequences = True),
    Dropout(0.5),
    LSTM(50, activation='sigmoid'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid') #prediksi nilai akhir kelas
])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 100)         200000    
                                                                 
 lstm_20 (LSTM)              (None, None, 200)         240800    
                                                                 
 lstm_21 (LSTM)              (None, None, 100)         120400    
                                                                 
 dropout_12 (Dropout)        (None, None, 100)         0         
                                                                 
 lstm_22 (LSTM)              (None, 50)                30200     
                                                                 
 dropout_13 (Dropout)        (None, 50)                0         
                                                                 
 dense_8 (Dense)             (None, 128)              

In [164]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [165]:
history = model.fit(x_train, y_train, epochs=100, 
                    validation_data=(x_test, y_test), verbose=1)

Epoch 1/100
75/75 [==============================] - 47s 557ms/step - loss: 0.4653 - accuracy: 0.8312 - val_loss: 0.4540 - val_accuracy: 0.8200
Epoch 2/100
75/75 [==============================] - 43s 568ms/step - loss: 0.4367 - accuracy: 0.8370 - val_loss: 0.4501 - val_accuracy: 0.8200
Epoch 3/100
75/75 [==============================] - 41s 550ms/step - loss: 0.4327 - accuracy: 0.8370 - val_loss: 0.4456 - val_accuracy: 0.8200
Epoch 4/100
75/75 [==============================] - 41s 546ms/step - loss: 0.4117 - accuracy: 0.8370 - val_loss: 0.4306 - val_accuracy: 0.8200
Epoch 5/100
75/75 [==============================] - 41s 540ms/step - loss: 0.3762 - accuracy: 0.8370 - val_loss: 0.3769 - val_accuracy: 0.8200
Epoch 6/100
75/75 [==============================] - 43s 570ms/step - loss: 0.3481 - accuracy: 0.8370 - val_loss: 0.2350 - val_accuracy: 0.8200
Epoch 7/100
75/75 [==============================] - 42s 567ms/step - loss: 0.1613 - accuracy: 0.9304 - val_loss: 0.1858 - val_accuracy:

In [ ]:
model.evaluate(x_train, y_train)